In [1]:
import dlib
import cv2 as cv
from tqdm import tqdm
from tqdm import tnrange
from IPython.display import Image
import numpy as np


In [2]:
pNum = "T000001"

proj = "/Users/ivan/Desktop/ALL/Code/PyProduct/GitProj/gproj/code/SecurityAI_Round1"
pimg = f"{proj}/Data/images"
pmdl = f"{proj}/Model"
pout = f"{proj}/Out/{pNum}"

In [3]:
modelv = "_68"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(f"{pmdl}/shape_predictor{modelv}_face_landmarks.dat")

In [4]:
# st, ed = 1, 100
st, ed = 1, 712


In [5]:
def f_xy(images):
    cv_face = detector(cv.cvtColor(images, cv.COLOR_BGR2GRAY), 1)
    if cv_face:
        result = []
        for face in cv_face:
            shape = predictor(images, face)
            for pt in shape.parts():
                result.append((pt.y, pt.x))
    else:
        result = [[56, 56]]
    return result

%time f_xy(cv.imread(f"{pimg}/00001.jpg"))[0]


CPU times: user 6.83 ms, sys: 984 µs, total: 7.82 ms
Wall time: 7.01 ms


(69, 19)

In [6]:
def f_lcos(f1, f2):
    df1 = [np.average(i) for i in f1]
    df2 = [np.average(i) for i in f2]
    return 1 - np.dot(df1/np.linalg.norm(df1, 2), df2/np.linalg.norm(df2, 2))

print("SAME:%.6f" % f_lcos(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00001.jpg")))
print("DIFF:%.6f" % f_lcos(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00002.jpg")))
print("DIFF:%.6f" % f_lcos(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00406.jpg")))

%time f_lcos(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00002.jpg"))


SAME:0.000000
DIFF:0.073383
DIFF:0.004231
CPU times: user 2.19 ms, sys: 222 µs, total: 2.41 ms
Wall time: 2.28 ms


0.07338285522785237

In [7]:
def f_lf1(f1, f2):
    return np.sqrt(np.sum(np.square(f1 - f2)))

print("SAME:%.6f" % f_lf1(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00001.jpg")))
print("DIFF:%.6f" % f_lf1(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00002.jpg")))
print("DIFF:%.6f" % f_lf1(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00406.jpg")))

%time f_lf1(cv.imread(f"{pimg}/00001.jpg"), cv.imread(f"{pimg}/00002.jpg"))


SAME:0.000000
DIFF:1974.158302
DIFF:1993.636627
CPU times: user 484 µs, sys: 176 µs, total: 660 µs
Wall time: 531 µs


1974.1583016566833

In [ ]:
"""
简单思路
    根据L2距离找出最近人脸；
    根据clip限制取结果；
    适当压缩再复原；
    # 是否可以考虑放大；
"""

def frun(tpath, time):
    for i in tnrange(st, ed+1, desc=f"{time+1}"):
        pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
        op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"

        img = cv.imread(pi)
        imgj, min_p012 = 1, 99999.0

        for imgi in range(st, ed+1, 1):
            imgx = cv.imread(tpath+f"00000{imgi}"[-5:]+".jpg")
            p012 = f_lf1(img, imgx)

            if p012 < min_p012 and imgi != i:
                imgj, min_p012 = imgi, p012

        # print(f"{i},{imgj}, Avg:{min_p012:.{6}}")

        img_o = cv.imread(tpath+f"00000{imgj}"[-5:]+".jpg")
        img_o = np.clip(img_o, img-25.5, img+25.5)

        cv.imwrite(op, img_o)

for n in tnrange(50):
    if n == 0:
        frun(f"{pimg}/", n)
    else:
        frun(f"{pout}/images/", n)

# 1, 37.7798
# 2, 41.2963
# 3, 40.8738
# 4, 40.2497
# 5, 40.4047
# 6, ?
# 7, -



In [ ]:
Image(filename=f"{pimg}/00001.jpg", width=300, height=300)

In [ ]:
Image(filename=f"{pout}/images/00001.jpg", width=300, height=300)

```
cv2.bitwise_and(), cv2.bitwise_not(), cv2.bitwise_or(), cv2.bitwise_xor()
img_o[y, x] = cv.bitwise_xor(img_o[y, x], img_o[y, x])[:,0]
img_o[y-n:y+n, x-n:x+n] = cv.bitwise_xor(img_o[y-n:y+n, x-n:x+n], img_o[y-n:y+n, x-n:x+n])
```


BACKUP

In [ ]:
# for imgi in range(1, 712):
#     img1 = cv.imread(f"{pimg}/"+f"00000{imgi}"[-5:]+".jpg")
# print(f"{x0},{n},{imgi},Avg01:{np.mean(p01):.{6}},Avg02:{np.mean(p02)*2/1000:.{6}}")

#         p0, p1 = f_xy(img), f_xy(img_o)
#         if p0:
#             pn = 0
#             for pi in p0:
#                 if pi in p1:
#                     pn += 1
#             p01.append(pn)


# x0 = 6
# n = 2

# for i in tnrange(st, ed+1):
#     pi = f"{pimg}/"+f"00000{i}"[-5:]+".jpg"
#     op = f"{pout}/images/"+f"00000{i}"[-5:]+".jpg"
    
#     img = cv.imread(pi)
#     imgj, min_p012 = 1, 99999.0
    
#     for imgi in tnrange(st, ed+1, 1):
#         img1 = cv.imread(f"{pimg}/"+f"00000{imgi}"[-5:]+".jpg")
#         img_o = img.copy()
        
#         for if_xy in f_xy(img):
#             y, x = if_xy
#             if 0 <= x < 112 and 0 <= y < 112:
#                 img_o[y-n:y+n, x-n:x+n] = cv.bitwise_xor(img_o[y-n:y+n, x-n:x+n], img1[y-n:y+n, x-n:x+n])
        
#         img_o = cv.resize(img_o, (x0, x0), cv.INTER_LINEAR)
#         img_o = cv.resize(img_o, (112, 112), cv.INTER_LINEAR)
#         img_o = np.clip(img_o, img-25.5, img+25.5)

#         p012 = f_lcos(img, img_o)
        
#         if p012 < min_p012 and imgi != i:
#             imgj, min_p012 = imgi, p012
    
#     print(f"{i},{imgj}, Avg:{min_p012:.{6}}")
    
#     img1 = cv.imread(f"{pimg}/"+f"00000{imgj}"[-5:]+".jpg")
#     img_o = img.copy()
#     for if_xy in f_xy(img):
#         y, x = if_xy
#         if 0 <= x < 112 and 0 <= y < 112:
#             img_o[y-n:y+n, x-n:x+n] = cv.bitwise_xor(img_o[y-n:y+n, x-n:x+n], img1[y-n:y+n, x-n:x+n])
#     img_o = cv.resize(img_o, (x0, x0), cv.INTER_LINEAR)
#     img_o = cv.resize(img_o, (112, 112), cv.INTER_LINEAR)
#     img_o = np.clip(img_o, img-25.5, img+25.5)
#     cv.imwrite(op, img_o)





In [ ]:
# cosin相似度（余弦相似度）
# https://blog.csdn.net/weixin_39121325/article/details/84187453
# 7941a200e221df958f3fd182e9d687845b387c897b783493
